# Deploying MCP Server from the AWS AI Agents Marketplace on Amazon Bedrock AgentCore Runtime - OAuth Inbound Authentication

## Overview

In this tutorial we will learn how to host MCP (Model Context Protocol) servers from the AWS AI Agents Marketplace on Amazon Bedrock AgentCore Runtime. 



### Tutorial Details

| Information         | Details                                                   |
|:--------------------|:----------------------------------------------------------|
| Tutorial type       | Hosting Tools                                             |
| Tool type           | MCP server                                                |
| Tutorial components | Hosting MCP server from the AWS AI Agents Marketplace on AgentCore Runtime                  |
| Tutorial vertical   | Cross-vertical                                            |
| Example complexity  | Easy                                                      |
| SDK used            | Amazon BedrockAgentCore Python SDK and MCP               |

### Tutorial Architecture

In this tutorial we will describe how to deploy an MCP server from the AWS AI Agents Marketplace to AgentCore runtime.

For demonstration purposes, we will use the [Tavily MCP Server](https://aws.amazon.com/marketplace/pp/prodview-twjga5bwmoszq)

<div style="text-align:left">
    <img src="images/hosting_mcp_server.png" width="60%"/>
</div>

### Tutorial Key Features

* Hosting MCP servers from the marketplace on Amazon Bedrock AgentCore Runtime
* Invoking deployed MCP servers with authentication


## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials configured
* Amazon Bedrock AgentCore SDK
* MCP (Model Context Protocol) library

In [ ]:
!pip install --force-reinstall -U -r dev-requirements.txt --quiet

In [4]:
from bedrock_agentcore_starter_toolkit import Runtime
from bedrock_agentcore_starter_toolkit.operations.runtime import destroy_bedrock_agentcore
from boto3.session import Session
from pathlib import Path
import os
import sys

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

sys.path[0]: /Users/hpoonawa/Documents/GitHub/amazon-bedrock-agents-healthcare-lifesciences/agents_catalog


In [5]:
boto_session = Session()
region = boto_session.region_name

ssm_client = boto_session.client('ssm', region_name=region)
secrets_client = boto_session.client('secretsmanager', region_name=region)
agentcore_control_client = boto_session.client("bedrock-agentcore-control", region_name=region)
ssm_client = boto_session.client('ssm', region_name=region)

tool_name = "mcp_server_agentcore"

## Understanding MCP (Model Context Protocol)

MCP is a protocol that allows AI models to securely access external data and tools. Key concepts:

* **Tools**: Functions that the AI can call to perform actions
* **Streamable HTTP**: Transport protocol used by AgentCore Runtime
* **Session Isolation**: Each client gets isolated sessions via `Mcp-Session-Id` header
* **Stateless Operation**: Servers must support stateless operation for scalability

AgentCore Runtime expects MCP servers to be hosted on `0.0.0.0:8000/mcp` as the default path.


```

## Deploying MCP Server from the AWS AI Agents marketplace

Now let's deploy the Tavily MCP server. Follow the instructions listed [here](https://aws.amazon.com/marketplace/pp/prodview-twjga5bwmoszq) 

1. Subscribe to the marketplace MCP server 

 ![subscribe](../static/subscribe.png)

 2. Launch the marketplace MCP server with Bedrock AgentCore  

 ![launch](../static/launch.png)

 3. Host the MCP server on Bedrock AgentCore Runtime and Configure the Inbound Authentication (Default : Cognito)

  ![host](../static/host.png)

  4. Add additional configuration specific to the provider. For example, for the Tavily MCP server , add the Tavily API key environment variable. 

  ![config](../static/config.png)

Copy over the Agent ARN after deploying to Bedrock AgentCore Runtime

In [ ]:
AGENT_ARN = 'TO_UPDATE'

## Using Amazon Cognito for Authentication

AgentCore Runtime requires authentication. We use Amazon Cognito to provide JWT tokens for accessing our deployed MCP server.

Copy over the required cognito configuration parameters from your deployment

In [ ]:

POOL_ID = ''
CLIENT_ID = ''
DISCOVERY_URL = ''
USER_NAME = "testuser"
PASSWORD = "MyPassword123!"
TEMP_ADMIN_PASSWORD = "Temp123!"


Setting up Amazon Cognito user pool...
User Pool ID: us-east-1_COTd8DHeA
Client ID: 5k7f22h8nhr2gemgbldqi90d0i


In [ ]:
print("Setting up Amazon Cognito user pool...")
import boto3
boto_session = Session()
region = boto_session.region_name
# Initialize Cognito client
cognito_client = boto3.client("cognito-idp", region_name=region)
try:
    # Create User
    cognito_client.admin_create_user(
            UserPoolId=POOL_ID,
            Username=USER_NAME,
            TemporaryPassword=TEMP_ADMIN_PASSWORD,
            MessageAction="SUPPRESS",
    )
    # Set Permanent Password
    cognito_client.admin_set_user_password(
            UserPoolId=POOL_ID, Username=USER_NAME, Password=PASSWORD, Permanent=True
    )
    # Authenticate User and get Access Token
    auth_response = cognito_client.initiate_auth(
            ClientId=CLIENT_ID,
            AuthFlow="USER_PASSWORD_AUTH",
            AuthParameters={"USERNAME": USER_NAME, "PASSWORD": PASSWORD},
    )
    bearer_token = auth_response["AuthenticationResult"]["AccessToken"]
    refresh_token = auth_response["AuthenticationResult"]["RefreshToken"]
    cognito_config = { "pool_id": POOL_ID,
            "client_id": CLIENT_ID,
            "bearer_token": bearer_token,
            "refresh_token": refresh_token,
            "discovery_url": DISCOVERY_URL }
    print(f"User Pool ID: {cognito_config.get('pool_id', 'N/A')}")
    print(f"Client ID: {cognito_config.get('client_id', 'N/A')}")
except Exception as e:
    print(f"Error setting up Amazon Cognito user pool: {e}")

## Storing Configuration for Remote Access

Before we can invoke our deployed MCP server, let's store the Agent ARN and Cognito configuration in AWS Systems Manager Parameter Store and AWS Secrets Manager for easy retrieval:

In [11]:
import boto3
import json

ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    cognito_credentials_response = secrets_client.create_secret(
        Name='tavily_mcp_server/cognito/credentials',
        Description='Cognito credentials for MCP server',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials stored in Secrets Manager")
except secrets_client.exceptions.ResourceExistsException:
    secrets_client.update_secret(
        SecretId='tavily_mcp_server/cognito/credentials',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials updated in Secrets Manager")

agent_arn_response = ssm_client.put_parameter(
    Name='/tavily_mcp_server/runtime/agent_arn',
    Value=AGENT_ARN,
    Type='String',
    Description='Agent ARN for MCP server',
    Overwrite=True
)
print("✓ Agent ARN stored in Parameter Store")

print("\nConfiguration stored successfully!")
print(f"Agent ARN: {AGENT_ARN}")

✓ Cognito credentials stored in Secrets Manager
✓ Agent ARN stored in Parameter Store

Configuration stored successfully!
Agent ARN: arn:aws:bedrock-agentcore:us-east-1:942514891246:runtime/hosted_agent_tavily_oauth-yg0sIIGbLv


## Creating Remote Testing Client

Now let's create a client to test our deployed MCP server. This client will retrieve the necessary credentials from AWS and connect to the deployed server:

In [12]:
%%writefile my_mcp_client_remote.py
import asyncio
import boto3
import json
import sys
import base64
import time
from boto3.session import Session
from datetime import timedelta
import traceback

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

def get_refresh_token(client_id, refresh_token, region):
    """Refresh access token using refresh token"""
    cognito_client = boto3.client('cognito-idp', region_name=region)
    auth_response = cognito_client.initiate_auth(
        ClientId=client_id,
        AuthFlow='REFRESH_TOKEN_AUTH',
        AuthParameters={'REFRESH_TOKEN': refresh_token}
    )
    return auth_response['AuthenticationResult']['AccessToken']

def get_valid_token(bearer_token, client_id, refresh_token, region):
    """Check token expiry and refresh if needed"""
    try:
        payload = bearer_token.split('.')[1]
        payload += '=' * (4 - len(payload) % 4)
        decoded = json.loads(base64.b64decode(payload))
        
        current_time = int(time.time())
        if decoded['exp'] - current_time < 300:
            print("🔄 Token expiring soon, refreshing...")
            new_token = get_refresh_token(client_id, refresh_token, region)
            print("✓ Token refreshed successfully")
            return new_token
        
        return bearer_token
    except Exception as e:
        print("🔄 Invalid token, refreshing...", e)
        traceback.print_exc()
        return get_refresh_token(client_id, refresh_token, region)

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/tavily_mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='tavily_mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        refresh_token = parsed_secret['refresh_token']
        client_id = parsed_secret['client_id']
        print("✓ Retrieved credentials from Secrets Manager")
        
        # Validate and refresh token if needed
        bearer_token = get_valid_token(bearer_token, client_id, refresh_token, region)
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    if not agent_arn or not bearer_token:
        print("Error: AGENT_ARN or BEARER_TOKEN not retrieved properly")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")
    print("Headers configured ✓")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}")
                    print(f"   Description: {tool.description}")
                    if hasattr(tool, 'inputSchema') and tool.inputSchema:
                        properties = tool.inputSchema.get('properties', {})
                        if properties:
                            print(f"   Parameters: {list(properties.keys())}")
                    print()
                
                print(f"✅ Successfully connected to MCP server!")
                print(f"Found {len(tool_result.tools)} tools available.")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

Writing my_mcp_client_remote.py


## Testing Your Deployed MCP Server

Let's test our deployed MCP server using the remote client:

In [13]:
print("Testing deployed MCP server...")
print("=" * 50)
!python my_mcp_client_remote.py

Testing deployed MCP server...
Using AWS region: us-east-1
Retrieved Agent ARN: arn:aws:bedrock-agentcore:us-east-1:942514891246:runtime/hosted_agent_tavily_oauth-yg0sIIGbLv
✓ Retrieved credentials from Secrets Manager

Connecting to: https://bedrock-agentcore.us-east-1.amazonaws.com/runtimes/arn%3Aaws%3Abedrock-agentcore%3Aus-east-1%3A942514891246%3Aruntime%2Fhosted_agent_tavily_oauth-yg0sIIGbLv/invocations?qualifier=DEFAULT
Headers configured ✓

🔄 Initializing MCP session...
✓ MCP session initialized

🔄 Listing available tools...

📋 Available MCP Tools:
🔧 tavily_search
   Description: Search the web for real-time information about any topic. Use this tool when you need up-to-date information that might not be available in your training data, or when you need to verify current facts. The search results will include relevant snippets and URLs from web pages. This is particularly useful for questions about current events, technology updates, or any topic that requires recent information

## Invoking MCP Tools Remotely

Now let's create an enhanced client that not only lists tools but also invokes them to demonstrate the full MCP functionality:

In [2]:
%%writefile invoke_mcp_tools.py
import asyncio
import boto3
import json
import sys
import base64
import time
from boto3.session import Session
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

def get_refresh_token(client_id, refresh_token, region):
    """Refresh access token using refresh token"""
    cognito_client = boto3.client('cognito-idp', region_name=region)
    auth_response = cognito_client.initiate_auth(
        ClientId=client_id,
        AuthFlow='REFRESH_TOKEN_AUTH',
        AuthParameters={'REFRESH_TOKEN': refresh_token}
    )
    return auth_response['AuthenticationResult']['AccessToken']

def get_valid_token(bearer_token, client_id, refresh_token, region):
    """Check token expiry and refresh if needed"""
    try:
        payload = bearer_token.split('.')[1]
        payload += '=' * (4 - len(payload) % 4)
        decoded = json.loads(base64.b64decode(payload))
        
        current_time = int(time.time())
        if decoded['exp'] - current_time < 300:
            print("🔄 Token expiring soon, refreshing...")
            new_token = get_refresh_token(client_id, refresh_token, region)
            print("✓ Token refreshed successfully")
            return new_token
        
        return bearer_token
    except:
        print("🔄 Invalid token, refreshing...")
        return get_refresh_token(client_id, refresh_token, region)

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/tavily_mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='tavily_mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        refresh_token = parsed_secret['refresh_token']
        client_id = parsed_secret['client_id']
        print("✓ Retrieved credentials from Secrets Manager")
        
        # Validate and refresh token if needed
        bearer_token = get_valid_token(bearer_token, client_id, refresh_token, region)
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}: {tool.description}")
                
                print("\n🧪 Testing MCP Tools:")
                print("=" * 50)
                
                try:
                    print("\n➕ Testing tavily_search...")
                    add_result = await session.call_tool(
                        name="tavily_search",
                        arguments={"query": "which drugs are associated with breast cancer", "max_results": 3}
                    )
                    print(f"   Result: {add_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                try:
                    print("\n✖️  Testing tavily_extract...")
                    multiply_result = await session.call_tool(
                        name="tavily_extract",
                        arguments={"urls": ['https://www.cancerresearchuk.org/about-cancer/treatment/drugs'], "extract_depth": 'basic'}
                    )
                    print(f"   Result: {multiply_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                
                print("\n✅ MCP tool testing completed!")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

Writing invoke_mcp_tools.py


## Test Tool Invocation

Let's test our MCP tools by actually invoking them:

In [17]:
print("Testing MCP tool invocation...")
print("=" * 50)
!python invoke_mcp_tools.py

Testing MCP tool invocation...
Using AWS region: us-east-1
Retrieved Agent ARN: arn:aws:bedrock-agentcore:us-east-1:942514891246:runtime/hosted_agent_tavily_oauth-yg0sIIGbLv
✓ Retrieved credentials from Secrets Manager

Connecting to: https://bedrock-agentcore.us-east-1.amazonaws.com/runtimes/arn%3Aaws%3Abedrock-agentcore%3Aus-east-1%3A942514891246%3Aruntime%2Fhosted_agent_tavily_oauth-yg0sIIGbLv/invocations?qualifier=DEFAULT

🔄 Initializing MCP session...
✓ MCP session initialized

🔄 Listing available tools...

📋 Available MCP Tools:
🔧 tavily_search: Search the web for real-time information about any topic. Use this tool when you need up-to-date information that might not be available in your training data, or when you need to verify current facts. The search results will include relevant snippets and URLs from web pages. This is particularly useful for questions about current events, technology updates, or any topic that requires recent information.
🔧 tavily_extract: Extract and proc

## Connect the MCP server to your agent

First get the MCP server url and headers 

In [ ]:
from boto3.session import Session
import boto3
import sys
import json
from invoke_mcp_tools import get_valid_token

boto_session = Session()
region = boto_session.region_name
    
print(f"Using AWS region: {region}")
    
    
try:
    ssm_client = boto3.client('ssm', region_name=region)
    agent_arn_response = ssm_client.get_parameter(Name='/tavily_mcp_server/runtime/agent_arn')
    agent_arn = agent_arn_response['Parameter']['Value']
    print(f"Retrieved Agent ARN: {agent_arn}")

    secrets_client = boto3.client('secretsmanager', region_name=region)
    response = secrets_client.get_secret_value(SecretId='tavily_mcp_server/cognito/credentials')
    secret_value = response['SecretString']
    parsed_secret = json.loads(secret_value)
    bearer_token = parsed_secret['bearer_token']
    refresh_token = parsed_secret['refresh_token']
    client_id = parsed_secret['client_id']
    print("✓ Retrieved credentials from Secrets Manager")
        
    # Validate and refresh token if needed
    bearer_token = get_valid_token(bearer_token, client_id, refresh_token, region)
        
except Exception as e:
    print(f"Error retrieving credentials: {e}")
    sys.exit(1)
    
encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
print(f"\nmcp url: {mcp_url}")


Create the agent with the MCP client and Bedrock model

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client
import time

def get_all_mcp_tools_from_mcp_client(client):
    """Get all tools from MCP client with pagination."""
    more_tools = True
    tools = []
    pagination_token = None
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)
        tools.extend(tmp_tools)
        if tmp_tools.pagination_token is None:
            more_tools = False
        else:
            more_tools = True
            pagination_token = tmp_tools.pagination_token
    return tools



print("\n✅ Agent testing now!")
    # Create MCP client
client = MCPClient(
        lambda: streamablehttp_client(
            mcp_url, headers
        )
)
    
    # Create Bedrock model
model = BedrockModel(
        model_id="global.anthropic.claude-sonnet-4-20250514-v1:0",
        temperature=0.7,
        streaming=True,
)
    
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    agent = Agent(model=model, tools=all_tools)
                                
    start_time = time.time()
                                    
    try:
        result = agent('which drugs are associated with breast cancer')
        execution_time = time.time() - start_time
                                        
        print(f"\n⏱️  Execution time: {execution_time:.2f}s")
        print(f"🎯 Response: {result.message['content'][0]['text']}")

    except Exception as e:
        print(f"❌ Error executing agent: {str(e)}")

## Next Steps

Now that you have successfully deployed an MCP server to AgentCore Runtime, you can:

1. **Add More Tools**: Extend your MCP server with additional tools
2. **Custom Authentication**: Implement custom JWT authorizers
3. **Integration**: Integrate with other agents running on AgentCore

## Cleanup (Optional)

If you want to clean up the resources created during this tutorial, run the following cells:

In [ ]:
# print("🗑️  Starting cleanup process...")

# try:
#     ssm_client.delete_parameter(Name='/mcp_server/runtime/agent_arn')
#     print("✓ Parameter Store parameter deleted")
# except ssm_client.exceptions.ParameterNotFound:
#     print("ℹ️  Parameter Store parameter not found")

# try:
#     secrets_client.delete_secret(
#         SecretId='mcp_server/cognito/credentials',
#         ForceDeleteWithoutRecovery=True
#     )
#     print("✓ Secrets Manager secret deleted")
# except secrets_client.exceptions.ResourceNotFoundException:
#     print("ℹ️  Secrets Manager secret not found")

# print("\n✅ Cleanup completed successfully!")

In [ ]:
# destroy_bedrock_agentcore(
#     config_path=Path(".bedrock_agentcore.yaml"),
#     agent_name=tool_name,
#     delete_ecr_repo=True
# )

# 🎉 Congratulations!

Your MCP server from the AWS marketplace is now running on Amazon Bedrock AgentCore Runtime and ready for production use!

The deployed MCP server can now be integrated into larger AI applications and workflows!